In [2]:
# C:\Users\Joshoua\Pictures\Panasonic
# get 20 images from the above folder and convert them to greyscale, save them under the order they were processed in the png images folder
import os
import cv2
import random

def process_images(source_folder, destination_folder, num_images=20):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Initialize counter for naming images
    image_counter = 1

    # List to store file paths of images
    image_paths = []

    # Traverse through the source folder and its subfolders
    for root, dirs, files in os.walk(source_folder):
        for filename in files:
            # Check if the file is an image
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                image_paths.append(os.path.join(root, filename))

    # Shuffle the list of image paths
    random.shuffle(image_paths)

    # Process each image
    for image_path in image_paths[:num_images]:
        # Read the image
        image = cv2.imread(image_path)

        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Generate the destination file path
        destination_path = os.path.join(destination_folder, f"{image_counter}.png")

        # Write the grayscale image to the destination folder
        cv2.imwrite(destination_path, gray_image)

        # Increment the image counter
        image_counter += 1

        # Break if reached the desired number of images
        if image_counter > num_images:
            break

# Specify source folder containing images
source_folder = 'C:/Users/Joshoua/Pictures/Panasonic'

# Specify destination folder for processed images
destination_folder = 'png images'

# Number of random images to select
num_images = 20

# Call the function to process images
process_images(source_folder, destination_folder, num_images)



In [8]:
import os
import cv2
import random
import numpy as np

def intensity_to_value(patch):
    # Scale intensities to a value between 0 and 16
    return (patch / 256) * 16

def get_binary_vectors(images_folder, n, nVector):
    binary_vectors = []

    # Get the list of images in the images folder
    image_files = os.listdir(images_folder)

    # Load all images into memory
    images = [cv2.imread(os.path.join(images_folder, image_file), cv2.IMREAD_GRAYSCALE) for image_file in image_files]

    for _ in range(nVector):
        # Randomly select an image from the loaded images
        image = random.choice(images)

        # Get image dimensions
        height, width = image.shape

        # Randomly select the top-left corner of the patch
        start_x = random.randint(0, width - n)
        start_y = random.randint(0, height - n)

        # Extract the patch
        patch = image[start_y:start_y+n, start_x:start_x+n]

        # Convert intensity values to a range between 0 and 16
        scaled_patch = intensity_to_value(patch)

        # Convert each value in the patch to an 8-bit binary representation
        binary_patch = np.unpackbits(scaled_patch.astype(np.uint8))

        # Flatten the binary patch into a 1D array to create a binary vector
        binary_vector = binary_patch.flatten()

        # Store the binary vector in the list
        binary_vectors.append(binary_vector)

    return binary_vectors

# Define parameters
images_folder = 'png images'
patch_size = 4
iterations = 750000

# Get binary vectors
binary_vectors = get_binary_vectors(images_folder, patch_size, iterations)

In [56]:
def image_to_binary_vectors(image_path, patch_size):
    binary_vectors = []

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    height, width = image.shape

    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            # Extract the patch
            patch = image[y:y+patch_size, x:x+patch_size]

            # Convert pixel values to binary representation
            binary_patch = np.unpackbits(patch)

            # Append the binary patch to the list of binary vectors
            binary_vectors.append(binary_patch)

    return binary_vectors

test_image_array = image_to_binary_vectors("png images/19.png", 4)

In [63]:
def image_to_binary_vectors_scaled(image_path, patch_size):
    binary_vectors = []

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    height, width = image.shape

    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            # Extract the patch
            patch = image[y:y+patch_size, x:x+patch_size]

            scaled_patch = intensity_to_value(patch)

            binary_patch = np.unpackbits(scaled_patch.astype(np.uint8))

            # Flatten the binary patch into a 1D array to create a binary vector
            binary_vector = binary_patch.flatten()

            # Append the binary patch to the list of binary vectors
            binary_vectors.append(binary_patch)

    return binary_vectors

test_image_array = image_to_binary_vectors_scaled("png images/19.png", 4)

In [65]:
def create_image_from_binary_vectors(width, height, patch_size, binary_vectors):
    # Calculate the number of patches per row and column
    num_patches_x = width // patch_size
    num_patches_y = height // patch_size

    # Calculate the total number of patches
    total_patches = num_patches_x * num_patches_y

    # Check if the number of binary vectors matches the number of patches
    if len(binary_vectors) != total_patches:
        print("Error: Number of binary vectors does not match the number of patches.")
        return None

    # Create an empty image
    image = np.zeros((height, width), dtype=np.uint8)

    # Iterate over each patch and fill it with the corresponding binary vector
    for i in range(num_patches_y):
        for j in range(num_patches_x):
            # Calculate the start and end indices for the binary vector
            pixelIndex = (i*num_patches_x) + j

            # Extract the binary vector for the patch
            patch_binary_vector = binary_vectors[pixelIndex]

            # Reshape the binary vector to match the patch size
            patch_binary_vector = patch_binary_vector.reshape((patch_size*patch_size, 8))
            patch_binary_vector = np.packbits(patch_binary_vector)
            patch_binary_vector = patch_binary_vector.reshape((patch_size,patch_size))
            
            # Calculate the top-left corner coordinates of the patch
            x = j * patch_size
            y = i * patch_size

            # Fill the patch in the image with the binary values
            image[y:y+patch_size, x:x+patch_size] = patch_binary_vector * 255

    # Show the image

    cv2.imwrite("./output/test.png", image)

    return image

# Example usage:
width = 4000
height = 3000
patch_size = 4

image = create_image_from_binary_vectors(width, height, patch_size, test_image_array)
